#Imports

In [ ]:
#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
import base64
import sys
# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/TER'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive
%pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/TER


'/content/gdrive/My Drive/Colab Notebooks/TER'

#Classification

In [ ]:
import pandas as pd
V2 = pd.read_excel("ISAC_V2.xlsx")
V2.head()

,Unnamed: 0,Chip_Code,Chip_Type,French_Residence_Department,French_Region,Rural_or_urban_area,Allergy_Present,Severe_Allergy,Respiratory_Allergy,Food_Allergy,...,Min_Animaux,Moyenne_Alimentaires,Max_Alimentaires,Min_Alimentaires,Moyenne_Glycanes,Max_Glycanes,Min_Glycanes,Moyenne_Autres,Max_Autres,Min_Autres
0,WQW0008,202272574,ISAC_V2,51,6,1,0,0,0,0,...,0,0.00000,0.00,0,0.0,0.0,0,0.0,0.0,0.0
1,WQW0054,210553086,ISAC_V2,51,6,1,0,0,0,0,...,0,2.38800,19.25,0,0.0,0.0,0,0.0,0.0,0.0
2,WQW0059,210993011,ISAC_V2,51,6,1,0,0,0,0,...,0,0.00000,0.00,0,0.0,0.0,0,0.0,0.0,0.0
3,WQW0160,221402048,ISAC_V2,51,6,1,0,0,0,0,...,-1,0.12850,1.91,-1,0.0,0.0,0,0.0,0.0,0.0
4,WQW0189,223042355,ISAC_V2,67,6,0,0,0,0,0,...,0,0.04025,0.54,0,0.0,0.0,0,0.0,0.0,0.0


In [ ]:
target_1 = [
    "Allergy_Present",
    "Respiratory_Allergy",
    "Food_Allergy",
    "Venom_Allergy",
    "Severe_Allergy",
    "Type_of_Food_Allergy_Other",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA",
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom",
]

extra_columns = [
    "Chip_Type",
    "Chip_Code",
    "French_Region",
    "French_Residence_Department"
    ]

extra = ['History_of_food_anaphylaxis','First_degree_family_history_of_atopy', 'History_of_atopic_dematitis', 'History_of_mast_cell_disorders',
         'History_of_hymenoptera_venom_anaphylaxis','Mammalian_meat']
extra_1 = ["Conjunctivitis", "Oral_Syndrom", "Cardiovascular_symptoms", "Respiratory_symptoms", "Gastrointestinal_symptoms", "Food_polyallergies"]

Gina = ["GINA_(asthma)_0", "GINA_(asthma)_1", "GINA_(asthma)_2", "GINA_(asthma)_3", "GINA_(asthma)_4", "GINA_(asthma)_5", "GINA_(asthma)_9"]
inconnu = ["Treatment_of_athsma_9", "Treatment_of_rhinitis_9", "General_cofactors_9",
           "Age_of_onsets_9", "ARIA_(rhinitis)_9", "GINA_(asthma)_9", "Treatment_of_atopic_dematitis_9"]
Aria = ["ARIA_(rhinitis)_9", "ARIA_(rhinitis)_0", "ARIA_(rhinitis)_1", "ARIA_(rhinitis)_2", "ARIA_(rhinitis)_3", "ARIA_(rhinitis)_4", "ARIA_(rhinitis)_5"]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

targets = ["Allergy_Present", "Respiratory_Allergy", "Food_Allergy"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X = V2.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]

results = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = V2[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results).to_csv("results_V2_Allergie.csv", index=False)


🔍 Target: Allergy_Present | Model: RandomForest
📈 Accuracy: 0.9898
🎯 F1 (0): 0.9845 | F1 (1): 0.9923
📊 Precision: 0.9900 | AUC: 0.9986316568047338
📊 Confusion Matrix:
 [[258   0]
 [  1 522]]



🔍 Target: Allergy_Present | Model: XGBoost
📈 Accuracy: 0.9872
🎯 F1 (0): 0.9807 | F1 (1): 0.9904
📊 Precision: 0.9876 | AUC: 0.9996671597633136
📊 Confusion Matrix:
 [[258   0]
 [  1 522]]



🔍 Target: Allergy_Present | Model: LogisticRegression
📈 Accuracy: 0.9744
🎯 F1 (0): 0.9622 | F1 (1): 0.9807
📊 Precision: 0.9756 | AUC: 0.9956044713631795
📊 Confusion Matrix:
 [[241  17]
 [  5 518]]



🔍 Target: Allergy_Present | Model: SVM
📈 Accuracy: 0.9488
🎯 F1 (0): 0.9189 | F1 (1): 0.9626
📊 Precision: 0.9500 | AUC: 0.9649851791894608
📊 Confusion Matrix:
 [[232  26]
 [  7 516]]



🔍 Target: Respiratory_Allergy | Model: RandomForest
📈 Accuracy: 0.9859
🎯 F1 (0): 0.9811 | F1 (1): 0.9887
📊 Precision: 0.9864 | AUC: 0.9992610837438424
📊 Confusion Matrix:
 [[297   0]
 [  0 484]]


...


🔍 Target: Respiratory_Allergy | Model: SVM
📈 Accuracy: 0.9039
🎯 F1 (0): 0.8582 | F1 (1): 0.9272
📊 Precision: 0.9132 | AUC: 0.9480409433888498
📊 Confusion Matrix:
 [[235  62]
 [  7 477]]



🔍 Target: Food_Allergy | Model: RandomForest
📈 Accuracy: 0.9552
🎯 F1 (0): 0.9604 | F1 (1): 0.9482
📊 Precision: 0.9577 | AUC: 0.9904201434636217
📊 Confusion Matrix:
 [[451   0]
 [  0 330]]



🔍 Target: Food_Allergy | Model: XGBoost
📈 Accuracy: 0.9475
🎯 F1 (0): 0.9544 | F1 (1): 0.9381
📊 Precision: 0.9483 | AUC: 0.9859361733274777
📊 Confusion Matrix:
 [[451   0]
 [  0 330]]



🔍 Target: Food_Allergy | Model: LogisticRegression
📈 Accuracy: 0.9027
🎯 F1 (0): 0.9133 | F1 (1): 0.8886
📊 Precision: 0.9069 | AUC: 0.9524681598594642
📊 Confusion Matrix:
 [[423  28]
 [ 23 307]]



🔍 Target: Food_Allergy | Model: SVM
📈 Accuracy: 0.8565
🎯 F1 (0): 0.8801 | F1 (1): 0.8206
📊 Precision: 0.8612 | AUC: 0.9368979651588347
📊 Confusion Matrix:
 [[426  25]
 [ 84 246]]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

V2_sev = V2[V2["Allergy_Present"] == 1]
targets = ["Severe_Allergy"]
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}
X = V2_sev.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]
results_severe = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = V2_sev[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_severe.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_severe).to_csv("results_V2_severe.csv", index=False)


🔍 Target: Severe_Allergy | Model: RandomForest
📈 Accuracy: 0.9542
🎯 F1 (0): 0.8945 | F1 (1): 0.9707
📊 Precision: 0.9558 | AUC: 0.9838068181818184
📊 Confusion Matrix:
 [[118   0]
 [  0 405]]



🔍 Target: Severe_Allergy | Model: XGBoost
📈 Accuracy: 0.9446
🎯 F1 (0): 0.8762 | F1 (1): 0.9643
📊 Precision: 0.9460 | AUC: 0.9835449002217296
📊 Confusion Matrix:
 [[118   0]
 [  0 405]]



🔍 Target: Severe_Allergy | Model: LogisticRegression
📈 Accuracy: 0.9065
🎯 F1 (0): 0.7969 | F1 (1): 0.9390
📊 Precision: 0.9109 | AUC: 0.957055155210643
📊 Confusion Matrix:
 [[105  13]
 [  9 396]]



🔍 Target: Severe_Allergy | Model: SVM
📈 Accuracy: 0.6331
🎯 F1 (0): 0.5071 | F1 (1): 0.7064
📊 Precision: 0.8003 | AUC: 0.7994466001478197
📊 Confusion Matrix:
 [[  0 118]
 [  0 405]]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

# Données respiratoires
V2_res = V2[V2["Respiratory_Allergy"] == 1]

targets = ["Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X = V2_res.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]

results_res = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Boucle principale
for target in targets:
    y = V2_res[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        print(f"\n🔍 Target: {target} | Model: {model_name}")

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # Application de SMOTE sur les données d'entraînement
            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        # Entraînement final sur tout X (sans SMOTE ici, car prédiction globale)
        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_res.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_res).to_csv("results_V2_respiratoire.csv", index=False)


🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: RandomForest
📈 Accuracy: 0.7914
🎯 F1 (0): 0.8483 | F1 (1): 0.6639
📊 Precision: 0.7902 | AUC: 0.8670862926136362
📊 Confusion Matrix:
 [[328   0]
 [  0 156]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: XGBoost
📈 Accuracy: 0.8016
🎯 F1 (0): 0.8535 | F1 (1): 0.6903
📊 Precision: 0.8052 | AUC: 0.8659189551767676
📊 Confusion Matrix:
 [[328   0]
 [  0 156]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: LogisticRegression
📈 Accuracy: 0.8017
🎯 F1 (0): 0.8592 | F1 (1): 0.6594
📊 Precision: 0.8015 | AUC: 0.825619476010101
📊 Confusion Matrix:
 [[318  10]
 [ 59  97]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: SVM
📈 Accuracy: 0.7461
🎯 F1 (0): 0.8031 | F1 (1): 0.6305
📊 Precision: 0.7645 | AUC: 0.8063107638888889
📊 Confusion Matrix:
 [[315  13]
 [100  56]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Tree | Model: RandomForest
📈 Accuracy: 0.9050
🎯 F1 (0): 0.8931 | F1 (1): 0.9139
📊 Precision: 0.9121 | AUC: 0.963965663965664
📊 Confusion Matrix:
 [[215   0]
 [  0 269]]


...


🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Gram | Model: SVM
📈 Accuracy: 0.7851
🎯 F1 (0): 0.7752 | F1 (1): 0.7935
📊 Precision: 0.7903 | AUC: 0.8702999898652072
📊 Confusion Matrix:
 [[179  42]
 [ 54 209]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: RandomForest
📈 Accuracy: 0.9835
🎯 F1 (0): 0.9752 | F1 (1): 0.9877
📊 Precision: 0.9843 | AUC: 0.9958139379743833
📊 Confusion Matrix:
 [[167   0]
 [  0 317]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: XGBoost
📈 Accuracy: 0.9814
🎯 F1 (0): 0.9727 | F1 (1): 0.9859
📊 Precision: 0.9821 | AUC: 0.9966029708254268
📊 Confusion Matrix:
 [[167   0]
 [  0 317]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: LogisticRegression
📈 Accuracy: 0.9814
🎯 F1 (0): 0.9727 | F1 (1): 0.9858
📊 Precision: 0.9820 | AUC: 0.9984942036290322
📊 Confusion Matrix:
 [[164   3]
 [  0 317]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: SVM
📈 Accuracy: 0.6800
🎯 F1 (0): 0.4841 | F1 (1): 0.7677
📊 Precision: 0.6685 | AUC: 0.7011003468927894
📊 Confusion Matrix:
 [[ 27 140]
 [  5 312]]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

V2_food = V2[V2["Food_Allergy"] == 1]
targets = ["Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X=V2_food.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]
results_food = []

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = V2_food[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_food.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_food).to_csv("results_V2_food.csv", index=False)


🔍 Target: Type_of_Food_Allergy_Aromatics | Model: RandomForest
📈 Accuracy: 0.9776
🎯 F1 (0): 0.9887 | F1 (1): 0.0667
📊 Precision: 0.9662 | AUC: 0.8009273772204806
📊 Confusion Matrix:
 [[878   0]
 [  0  16]]



🔍 Target: Type_of_Food_Allergy_Aromatics | Model: XGBoost
📈 Accuracy: 0.9754
🎯 F1 (0): 0.9875 | F1 (1): 0.1900
📊 Precision: 0.9727 | AUC: 0.7526907001044931
📊 Confusion Matrix:
 [[878   0]
 [  0  16]]



🔍 Target: Type_of_Food_Allergy_Aromatics | Model: LogisticRegression
📈 Accuracy: 0.9351
🎯 F1 (0): 0.9662 | F1 (1): 0.0832
📊 Precision: 0.9684 | AUC: 0.7120885579937305
📊 Confusion Matrix:
 [[878   0]
 [ 15   1]]



🔍 Target: Type_of_Food_Allergy_Aromatics | Model: SVM
📈 Accuracy: 0.7998
🎯 F1 (0): 0.8872 | F1 (1): 0.0868
📊 Precision: 0.9737 | AUC: 0.7221068443051202
📊 Confusion Matrix:
 [[878   0]
 [ 16   0]]



🔍 Target: Type_of_Food_Allergy_Cereals_&_Seeds | Model: RandomForest
📈 Accuracy: 0.9608
🎯 F1 (0): 0.9800 | F1 (1): 0.0000
📊 Precision: 0.9403 | AUC: 0.7197986278178738
📊 Confusion Matrix:
 [[867   0]
 [  0  27]]


...


🔍 Target: Type_of_Food_Allergy_TPO | Model: SVM
📈 Accuracy: 0.8064
🎯 F1 (0): 0.8873 | F1 (1): 0.2910
📊 Precision: 0.9433 | AUC: 0.8249019607843138
📊 Confusion Matrix:
 [[848   0]
 [ 46   0]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: RandomForest
📈 Accuracy: 0.9172
🎯 F1 (0): 0.9560 | F1 (1): 0.2820
📊 Precision: 0.8992 | AUC: 0.8458188153310104
📊 Confusion Matrix:
 [[820   0]
 [  0  74]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: XGBoost
📈 Accuracy: 0.9183
🎯 F1 (0): 0.9560 | F1 (1): 0.4085
📊 Precision: 0.9070 | AUC: 0.8562282229965158
📊 Confusion Matrix:
 [[820   0]
 [  0  74]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: LogisticRegression
📈 Accuracy: 0.8681
🎯 F1 (0): 0.9271 | F1 (1): 0.2944
📊 Precision: 0.8830 | AUC: 0.7658101045296167
📊 Confusion Matrix:
 [[813   7]
 [ 60  14]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: SVM
📈 Accuracy: 0.7002
🎯 F1 (0): 0.8106 | F1 (1): 0.2641
📊 Precision: 0.8922 | AUC: 0.7194033101045296
📊 Confusion Matrix:
 [[820   0]
 [ 74   0]]


#Ne lancer pas cette partie, c pour la recherche des hyperparametres

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from xgboost import XGBClassifier

# Grille des hyperparamètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'gamma': [0, 1, 5]
}

# Données et targets
X = V1.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X = X.iloc[:, 1:]

targets = [
    "Allergy_Present",
    "Respiratory_Allergy",
    "Food_Allergy",
    "Venom_Allergy",
    "Severe_Allergy",
    "Type_of_Food_Allergy_Other",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA",
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom",
]

all_results = []

for target in targets:
    print(f"\n🔍 Recherche d'hyperparamètres pour le target: {target}")

    y = V1[target]
    filtered_X = X.copy()
    filtered_y = y.copy()

    if target in ["Severe_Allergy", "Respiratory_Allergy", "Food_Allergy", "Venom_Allergy"]:
        mask = V1["Allergy_Present"] == 1
        filtered_X = filtered_X[mask]
        filtered_y = filtered_y[mask]
    elif target.startswith("Type_of_Respiratory_Allergy_"):
        mask = V1["Respiratory_Allergy"] == 1
        filtered_X = filtered_X[mask]
        filtered_y = filtered_y[mask]
    elif target.startswith("Type_of_Food_Allergy_"):
        mask = V1["Food_Allergy"] == 1
        filtered_X = filtered_X[mask]
        filtered_y = filtered_y[mask]

    if len(filtered_y.unique()) < 2 or len(filtered_y) < 10:
        print(f" Pas assez de données pour {target}, on passe.")
        continue

    model = XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='f1',
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(filtered_X, filtered_y)


    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    best_params['Target'] = target
    best_params['Best_F1_Score'] = best_score

    all_results.append(best_params)

df_results = pd.DataFrame(all_results)
df_results = df_results[['Target'] + [col for col in df_results.columns if col != 'Target']]  # Remettre Target en premier
df_results.to_csv("xgboost_best_hyperparameters_V1.csv", index=False)


🔍 Recherche d'hyperparamètres pour le target: Allergy_Present
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Respiratory_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Food_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Venom_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Severe_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Type_of_Food_Allergy_Other
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Type_of_Respiratory_Allergy

#TOP Features

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import plotly.graph_objects as go

targets = [
    "Allergy_Present", "Respiratory_Allergy", "Food_Allergy", "Venom_Allergy",
    "Severe_Allergy", "Type_of_Food_Allergy_Other", "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree", "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach", "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA", "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram", "Type_of_Respiratory_Allergy_GINA",
    "Type_of_Food_Allergy_Aromatics", "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg", "Type_of_Food_Allergy_Fish", "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk", "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes", "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish", "Type_of_Food_Allergy_TPO", "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom", "Type_of_Venom_Allergy_IGE_Venom"
]

inconnu = ["Treatment_of_athsma_9", "Treatment_of_rhinitis_9", "General_cofactors_9",
           "Age_of_onsets_9", "ARIA_(rhinitis)_9", "GINA_(asthma)_9", "Treatment_of_atopic_dematitis_9"]

X = V2.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]

def plot_top_features(model, X_sub, y_sub, target):
    if len(np.unique(y_sub)) < 2:
        print(f"⚠️ Target '{target}' contient une seule classe ({np.unique(y_sub)}). Skip.")
        return

    model.fit(X_sub, y_sub)
    importances = model.feature_importances_
    top_indices = np.argsort(importances)[::-1][:10]
    features = X_sub.columns[top_indices]
    scores = importances[top_indices]

    fig = go.Figure(go.Bar(
        x=scores[::-1],
        y=features[::-1],
        orientation='h',
        text=[f"{s:.3f}" for s in scores[::-1]],
        textposition='outside'
    ))
    fig.update_layout(
        title=f"Top 10 Features pour la cible '{target}' (XGBoost)",
        xaxis_title="Importance",
        yaxis_title="Feature",
        width=800, height=500
    )
    fig.show()

for target in targets:
    X_sub = X.copy()
    y_sub = V2[target]
    model = XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False)
    plot_top_features(model, X_sub, y_sub, target)

⚠️ Target 'Type_of_Food_Allergy_Oral_Syndrom' contient une seule classe ([0]). Skip.


⚠️ Target 'Type_of_Venom_Allergy_IGE_Venom' contient une seule classe ([0]). Skip.
